# Data analysis workflow
![alt text](Selection_081.png)

# Planning

## Have a meeting with my boss where we discuss the project

![alt text](ew.png)


> ### We want to explore the price relationship btwn organics and conventional crops

# Data Collection

![alt text](usda.png)

# Data Management

### small project, static data

![alt text](Dropbox.png)

# Clean Data
![alt text](Data.png)


In [6]:
import pandas as pd
import numpy as np
from dateutil.parser import *
from datetime import *

from os.path import dirname, join

In [7]:
%pwd

'/home/brett/Documents/JupyterNotebooks/AgResearch'

In [8]:
%cd /home/brett/Dropbox/AgResearch/

/home/brett/Dropbox/AgResearch


In [9]:
Veggies2012 = pd.read_csv('Vegetables2012.csv',skiprows=1)

In [10]:
Veggies2013 = pd.read_csv('Vegetables2013.csv', skiprows=1)

In [11]:
#drop the year summary vars
#Veggies2012.columns.get_loc('2012');
Veggies2012 = Veggies2012.drop(Veggies2012.columns[17],1)
Veggies2013 = Veggies2013.drop(Veggies2013.columns[17],1)

In [12]:
#transform the data so dates are a column
Veggies2012 = pd.melt(Veggies2012, id_vars=['Commodity','Subgroup','Package','Organic/ Conventional','Terminal Market'],
       var_name='MonthYear', value_name='Price')

Veggies2013 = pd.melt(Veggies2013, id_vars=['Commodity','Subgroup','Package','Organic/ Conventional','Terminal Market'],
       var_name='MonthYear', value_name='Price')

In [13]:
#put em together
Ags = Veggies2012.append(Veggies2013)

In [14]:
DEFAULT = datetime(1,1,1)

In [15]:
#make sure month year is formatted correctly then convert it to datetime
#Veggies['MonthYear'].str.strip('\n')
Ags['MonthYear'] = Ags['MonthYear'].str.replace('-','20')
Ags['MonthYearForm'] = Ags['MonthYear'].apply(lambda x: parse(x,default=DEFAULT))

In [16]:
#convert price to number formatting
Ags['Price'] = pd.to_numeric(Ags['Price'],errors='coerce')

In [17]:
#want to have it so price for conventional and organic are columns
#couldnt quickly google how-to, so doing it 'by hand'
#first make the two columns, with price if there is one
Ags['Price_Conv'] = np.where(Ags['Organic/ Conventional']=='Conv',Ags['Price'],'NaN')
Ags['Price_Org'] = np.where(Ags['Organic/ Conventional']=='Org',Ags['Price'],'NaN')

In [18]:
#next convert new cols to numbers, my NaNs werent numbers
Ags['Price_Conv'] = pd.to_numeric(Ags['Price_Conv'],errors='coerce')
Ags['Price_Org'] = pd.to_numeric(Ags['Price_Org'],errors='coerce')

In [19]:
#back fill nans so one row has an entry for both cols
Ags['Price_Conv'] = Ags['Price_Conv'].fillna(method='bfill')
Ags['Price_Org'] = Ags['Price_Org'].fillna(method='bfill')

In [20]:
#price data in new cols is repeated and only the first is correct
#so put NaNs back where they are legit
def convOrg(row):
    if (row['Organic/ Conventional'] == 'Conv' and pd.isnull(row['Price'])):
        row['Price_Conv'] = 'NaN'
    if (row['Organic/ Conventional'] == 'Org' and pd.isnull(row['Price'])):
        row['Price_Org'] = 'NaN'
    return row

In [21]:
Ags = Ags.apply(convOrg,axis=1)

In [22]:
#drop now unwanted columns
#Ags.columns.get_loc('Organic/ Conventional');
Ags = Ags.drop(Ags.columns[[3,5,6]],1)

In [23]:
#drop dupes keep first entry bc thats the good one
Ags = Ags.drop_duplicates(['Commodity','MonthYearForm','Subgroup','Package','Terminal Market'], keep='first')

In [24]:
#next convert new cols to numbers, my NaNs werent numbers
Ags['Price_Conv'] = pd.to_numeric(Ags['Price_Conv'],errors='coerce')
Ags['Price_Org'] = pd.to_numeric(Ags['Price_Org'],errors='coerce')

In [25]:
#calculate the organic premium
Ags['Premium'] = Ags['Price_Org'] - Ags['Price_Conv']
Ags['PremiumAbs'] = (Ags['Price_Org'] - Ags['Price_Conv'])

In [26]:
Ags = Ags.set_index(['MonthYearForm'])

In [27]:
#replace nan with NaN string so the list of Uniquely doesnt have NaNs in it
Ags.Commodity = Ags.Commodity.fillna('NaN')
Ags.Subgroup = Ags.Subgroup.fillna('NaN')
Ags.Package = Ags.Package.fillna('NaN')
Ags['Terminal Market'] = Ags['Terminal Market'].fillna('NaN')

In [28]:
#make a unique variable so convert to list to be used in the dropdown for the graph
Ags['Uniquely'] = Ags.Commodity + "; " + Ags.Subgroup + "; " + Ags.Package+ "; " + Ags['Terminal Market']

In [31]:
Ags.head()

,Commodity,Subgroup,Package,Terminal Market,Price_Conv,Price_Org,Premium,PremiumAbs,Uniquely
MonthYearForm,,,,,,,,,
2012-01-01,Artichoke (SF only),Globe,"cartons, 24s",San Fran,28.58,37.34,8.76,8.76,"Artichoke (SF only); Globe; cartons, 24s; San ..."
2012-01-01,Cabbage,Round green type,"40 lb cartons, medium",Atlanta,NaN,33.95,NaN,NaN,"Cabbage; Round green type; 40 lb cartons, medi..."
2012-01-01,Cabbage,Round green type,"45 lb cartons, medium",Atlanta,14.85,33.95,19.10,19.10,"Cabbage; Round green type; 45 lb cartons, medi..."
2012-01-01,Cabbage,Round green type,"45 lb cartons, 18-24s",San Fran,14.85,24.97,10.12,10.12,"Cabbage; Round green type; 45 lb cartons, 18-2..."
2012-01-01,Carrots,Not specified,25 lb sacks loose,Atlanta,12.50,24.97,12.47,12.47,Carrots; Not specified; 25 lb sacks loose; Atl...


In [29]:
ulist = Ags.Uniquely.tolist()

In [30]:
#change pwd to where I want to pickle
%cd Documents/JupyterNotebooks/AgResearch/OrganicGraph/

[Errno 2] No such file or directory: 'Documents/JupyterNotebooks/AgResearch/OrganicGraph/'
/home/brett/Dropbox/AgResearch


In [26]:
Ags.to_pickle('Ags.pkl')

In [29]:
ulist

['Artichoke (SF only); Globe; cartons, 24s; San Fran',
 'Cabbage; Round green type; 40 lb cartons, medium; Atlanta',
 'Cabbage; Round green type; 45 lb cartons, medium; Atlanta',
 'Cabbage; Round green type; 45 lb cartons, 18-24s; San Fran',
 'Carrots; Not specified; 25 lb sacks loose; Atlanta',
 'Carrots; Topped; 25 lb sacks loose; San Fran',
 'Cauliflower; White; cartons, film wrapped, 12s; Atlanta',
 'Cauliflower; White; cartons, film wrapped, 12s; San Fran',
 'Greens; Swiss Chard, mixed colors; cartons bunched, 24s; Atlanta',
 'Greens; Swiss Chard, mixed colors; cartons bunched, 24s; San Fran',
 'Lettuce; Green leaf; cartons, 24s; Atlanta',
 'Lettuce; Green leaf; cartons, 24s; San Fran',
 'Lettuce; Romaine; cartons, 24s; Atlanta',
 'Lettuce; Romaine; cartons, 24s; San Fran',
 'Mesculin Mix; NaN; 3 lb cartons; Atlanta',
 'Mesculin Mix; NaN; 3 lb cartons; San Fran',
 'Onions, Dry; Yellow, Hybrid; All packages - no 40 lb cartons, JBO; Atlanta',
 'Onions, Dry; Yellow, Hybrid; 40 lb car